In [ ]:
import warnings
warnings.filterwarnings('ignore')
#-----------------------------------
import time
import pandas as pd
import numpy as np
from datetime import datetime
import logging
#-----------------------------------
import sqlalchemy
# from sqlalchemy import create_engine,text
#-----------------------------------

In [ ]:
user="sa"
password="SA@123"
server="Aditya123\SQLSERVER22DEV" # SELECT @@SERVERNAME;
db="Local_DB"
# ms_sql_db_Engine = sqlalchemy.create_engine(f"mssql+pyodbc://{user}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server",fast_executemany=True)
ms_sql_db_Engine = sqlalchemy.create_engine(f"mssql+pyodbc://{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server",fast_executemany=True)
ms_sql_db_Connection = ms_sql_db_Engine.connect()

#-----------------------------------
ms_sql_db_Connection.close()
ms_sql_db_Engine.dispose()
#-----------------------------------

In [ ]:
def ms_sql_db_Sonata():
    #-----------------------------------
    user="SFPL_Connect"
    password="$%n5bF33%X"
    server="172.17.130.216"
    db="Sonata_Connect"
    db="Sonata_Connect_Test"
    db="CRN_GEN"
    #-----------------------------------
    ms_sql_db_Engine = sqlalchemy.create_engine(f"mssql+pyodbc://{user}:{password}@{server}/{db}?driver=ODBC+Driver+17+for+SQL+Server",fast_executemany=True)
    ms_sql_db_Connection = ms_sql_db_Engine.connect()
    #-----------------------------------
    return ms_sql_db_Engine,ms_sql_db_Connection

In [ ]:
def pg_sql_db_Markytics():
    #-----------------------------------
    user="postgres"
    password="Markytics1234"
    server="34.100.140.42"
    db="sonata_voicebot"
    #-----------------------------------
    pg_sql_db_Engine = sqlalchemy.create_engine(f"postgresql+psycopg2://{user}:{password}@{server}:5432/{db}")
    pg_sql_db_Connection = pg_sql_db_Engine.connect()
    #-----------------------------------
    return pg_sql_db_Engine,pg_sql_db_Connection

In [ ]:
# fetch records :
sql_query=F'''
SET NOCOUNT ON;
EXEC SP_MappingCallingUsersToRegion @RegionID=1012, @UserID=17861;
SET NOCOUNT OFF;
'''
df=pd.read_sql_query(sql_query,ms_sql_db_Engine);df

In [ ]:
# fetch records :
sql_query=F'''
SELECT TOP 5 * FROM Day_Wise_Summarised_final;
'''
df=pd.read_sql_query(sql_query,ms_sql_db_Engine);df

In [ ]:
# send records to DB :
df.to_sql(name='table_name', con=ms_sql_db_Engine, index=False, if_exists='append', chunksize=10000)
df.to_sql(name='table_name', con=ms_sql_db_Engine, index=False, if_exists='replace', chunksize=10000)

In [ ]:
# DDL / DML in DB :
try:
    transaction = ms_sql_db_Connection.begin()
    query = sqlalchemy.text('TRUNCATE TABLE Sonata_Connect..table_name;')
    ms_sql_db_Connection.execute(query)
    transaction.commit()
except Exception as e:
    print(f"Error occurred: {e}")
    transaction.rollback()
else:
    print("Query executed successfully.")
finally:
    transaction.close()
    ms_sql_db_Connection.close()

In [ ]:
# DDL / DML in DB :
with ms_sql_db_Connection.begin():  # Automatically manages commit and rollback
    query = sqlalchemy.text('TRUNCATE TABLE Sonata_Connect..table_name;')
    ms_sql_db_Connection.execute(query)